## Instalando pacotes

In [ ]:
!pip install googletrans

In [ ]:
!python -m textblob.download_corpora

## Importando pacotes

In [3]:
import pandas as pd
import numpy as np
import time
import googletrans as Translator
from textblob import TextBlob

## Lendo o gabarito

In [4]:
df_gabarito = pd.read_csv('/content/drive/MyDrive/UEA - Pós Ciência de Dados/2. Programação para Ciência de Dados/ds-proc-dados-uea/artefatos/dados/gabarito.csv', index_col=0)

## Traduzindo os comentários

In [5]:
translator = Translator.Translator()

def traduzir_googletrans(comentario) :
  try :
    comentario_traduzido = translator.translate(comentario, dest='en').text
  except : 
    comentario_traduzido = comentario
  finally :
    print(comentario_traduzido) # Apenas para acompanhamento da tradução
    time.sleep(1)
    return comentario_traduzido

In [6]:
def traduzir_textblob(comentario) :
    try :
        comentario_traduzido = TextBlob(comentario).translate(to='us')
    except:
        comentario_traduzido = comentario
    finally:
        print(comentario_traduzido) # Apenas para acompanhamento da tradução
        time.sleep(1)
        return comentario_traduzido

In [ ]:
sr_sentences_textblob = df_gabarito['sentence'].apply(traduzir_textblob)

In [ ]:
sr_sentences_googletrans = df_gabarito['sentence'].apply(traduzir_googletrans)

In [ ]:
# Salvando as traduções em arquivo csv por segurança
#sr_sentences_textblob.to_csv('/content/drive/MyDrive/UEA - Pós Ciência de Dados/2. Programação para Ciência de Dados/ds-proc-dados-uea/artefatos/dados/200sentencas-textblob.csv')
#sr_sentences_googletrans.to_csv('/content/drive/MyDrive/UEA - Pós Ciência de Dados/2. Programação para Ciência de Dados/ds-proc-dados-uea/artefatos/dados/200sentencas-googletrans.csv')

In [ ]:
# Lendo os arquivos traduzidos caso necessários
#df_sentences_textblob = pd.read_csv('/content/drive/MyDrive/UEA - Pós Ciência de Dados/2. Programação para Ciência de Dados/ds-proc-dados-uea/artefatos/dados/200sentencas-textblob.csv', index_col=0)
#df_sentences_googletrans = pd.read_csv('/content/drive/MyDrive/UEA - Pós Ciência de Dados/2. Programação para Ciência de Dados/ds-proc-dados-uea/artefatos/dados/200sentencas-googletrans.csv', index_col=0)

## Calculando Subjetividade de ambas as traduções

In [ ]:
#Função para calcular a subjetividade de um comentário
def calcular_subjetividade(comentario) :
  try :
    comentario_blob = TextBlob(comentario)
    return comentario_blob.sentiment.subjectivity
  except:
    return np.nan

In [ ]:
# Transformando as Series em DataFrames
df_sentences_textblob = sr_sentences_textblob.to_frame()
df_sentences_googletrans = sr_sentences_googletrans.to_frame()

In [ ]:
# Calculando a subjetividade de ambas as traduções
df_sentences_textblob['subjetividade'] = df_sentences_textblob['sentence'].apply(calcular_subjetividade)
df_sentences_googletrans['subjetividade'] = df_sentences_googletrans['sentence'].apply(calcular_subjetividade)

## Definindo se a sentença é subjetiva ou objetiva

In [ ]:
SUBJETIVIDADE = 0.28  # Objetivo < SUBJETIVIDADE | Opinativo >= SUBJETIVIDADE
POLARIDADE_N = 0.15   # Negativo < POLARIDADE_N 
POLARIDADE_P = 0.2    # Positivo >= POLARIDADE_P
                      # [(Neutro >= POLARIDADE_N) e (Neutro < POLARIDADE_P)]

In [ ]:
def definir_subjetividade(subjetividade) :
  if subjetividade < SUBJETIVIDADE :
    return 0
  else :
    return 1

In [ ]:
df_sentences_textblob['subjetividade'] = df_sentences_textblob['subjetividade'].apply(definir_subjetividade)
df_sentences_googletrans['subjetividade'] = df_sentences_googletrans['subjetividade'].apply(definir_subjetividade)

## Calculando Medida-F de ambas as análises

In [ ]:
def precisao(analise, gabarito) :
  qtd_acerto = 0
  qtd_total = 0
  if (len(analise) == len(gabarito)) :
    for i in range(len(analise)) :
      if (analise[i]==1) :
        qtd_total = qtd_total + 1 # Conta a quantidade de sentenças subjetivas indicadas pelo textblob
        if (analise[i]==gabarito[i]) : 
          qtd_acerto = qtd_acerto + 1 # Conta o total de acertos que o textblob realizou referente as sentenças subjetivas do gabarito
    return qtd_acerto / qtd_total

In [ ]:
def revocacao(analise, gabarito) :
  qtd_acerto = 0
  qtd_total = 0
  if (len(analise) == len(gabarito)) :
    for i in range(len(analise)) :
      if (gabarito[i]==1) :
        qtd_total = qtd_total = qtd_total + 1 # Conta a quantidade de sentenças subjetivas indicadas pelo gabarito
        if (analise[i]==1) :
          qtd_acerto = qtd_acerto + 1 # Conta o total de acerto que o textblob realizou referente as sentenças subjetivas do gabarito 
      return qtd_acerto / qtd_total


In [ ]:
def medidaf(precisao, revocacao) :
  return (2 * precisao * revocacao) / (precisao + revocacao)

In [ ]:
precisao_googletrans = precisao(list(df_sentences_googletrans['subjetividade']), list(df_gabarito['subjetividade']))
revocacao_googletrans = revocacao(list(df_sentences_googletrans['subjetividade']), list(df_gabarito['subjetividade']))
medidaf_googletrans = medidaf(precisao_googletrans, revocacao_googletrans)

precisao_textblob = precisao(list(df_sentences_textblob['subjetividade']), list(df_gabarito['subjetividade']))
revocacao_textblob = revocacao(list(df_sentences_textblob['subjetividade']), list(df_gabarito['subjetividade']))
medidaf_textblob = medidaf(precisao_textblob, revocacao_textblob)

In [ ]:
print("\n\nMedidadeF referente à subjetividade das sentenças traduzidas pelo GOOGLETRANS\n")
print("Precisão: {0} \nRevocação: {1} \nMedida-F: {2}".format(precisao_googletrans, revocacao_googletrans, medidaf_googletrans))

print("\n\nMedidaF referente à subjetividade das senteças traduzidas pelo TEXTBLOB\n")
print("Precisão: {0} \nRevocação: {1} \nMedida-F: {2}".format(precisao_googletrans, revocacao_googletrans, medidaf_googletrans))



MedidadeF referente à subjetividade das sentenças traduzidas pelo GOOGLETRANS

Precisão: 0.8551724137931035 
Revocação: 1.0 
Medida-F: 0.9219330855018588


MedidaF referente à subjetividade das senteças traduzidas pelo TEXTBLOB

Precisão: 0.8551724137931035 
Revocação: 1.0 
Medida-F: 0.9219330855018588
